In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
cd /content/drive/MyDrive/textGCN-code/G1_Code - Copy

/content/drive/MyDrive/textGCN-code/G1_Code - Copy


In [22]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [23]:
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score

# Data loading

In [24]:
d = pd.read_csv('./data/mr.sbert_emb.csv', header = None)
d.head()

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.032374,-0.038591,-0.009931,0.019342,0.066781,0.045675,0.097774,-0.018270,-0.030173,-0.090040,...,0.063426,-0.046464,-0.025931,-0.043174,-0.072244,0.025198,0.029846,0.003866,0.064594,-0.022676
1,0.013990,0.035276,-0.053836,0.045904,-0.030362,-0.024570,0.091804,0.043476,0.006043,0.047313,...,-0.023293,-0.046520,-0.020535,0.057097,-0.031597,0.077903,0.120587,-0.046387,0.116128,-0.028451
2,-0.090458,-0.027348,0.037921,-0.022743,-0.066227,-0.039529,-0.041533,-0.067170,0.101122,-0.027909,...,0.059399,-0.044684,-0.043817,0.084780,-0.029977,0.057045,0.093044,0.034575,0.064776,0.012819
3,-0.018563,0.057456,0.052753,0.002394,-0.021295,-0.046604,0.035935,0.006271,-0.071811,0.013771,...,0.068337,0.047285,-0.042631,-0.019054,-0.026286,-0.006568,0.012489,0.013544,0.055660,0.016322
4,0.065692,-0.041605,0.122817,0.050609,0.002123,0.044241,-0.027913,-0.063429,-0.078185,-0.038174,...,0.072214,-0.015732,-0.066744,0.067960,0.013191,0.051593,0.018374,0.013251,0.014035,0.066022


In [55]:
docs = []
labels = []
f = open('./data/mr.txt', 'r', encoding="utf8")
lines = f.readlines()
for line in lines:
    temp = line.split("\t")
    labels.append(temp[2])
f.close()

f_labels = []
for l in labels:
  f_labels.append(int(l.replace("\n","")))   # MR dataset
  # f_labels.append(l.replace("\n",""))  # sameval dataset
# f_labels = np.array(labels)
print(f_labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
le = preprocessing.LabelEncoder()
n_labels = le.fit_transform(f_labels)
un_labels = np.unique(n_labels)
un_labels

array([0, 1])

In [ ]:
print((d.values).shape)
print(len(n_labels))

(10662, 384)
10662


In [ ]:
x_all = d.values
y_all = np.array(n_labels)


In [ ]:
# split train into train and test ----> by default, data first shuffled and then divided
X_train, X_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.3333, random_state=1, stratify=y_all)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(7108, 384) (3554, 384)
(7108,) (3554,)


#Implementing basic ML classifiers

In [ ]:
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(max_iter= 200)
LR_model.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [ ]:
LR_yhat = LR_model.predict(X_test)
LR_score = accuracy_score(y_test, LR_yhat)
print("Accuracy on movie review dataset using LR together: %.3f" % (LR_score*100))

Accuracy on movie review dataset using LR together: 71.867


In [ ]:
from sklearn.svm import SVC
SVM_model = SVC()
SVM_model.fit(X_train, y_train)

SVC()

In [ ]:
svm_yhat = SVM_model.predict(X_test)
svm_score = accuracy_score(y_test, svm_yhat)
print("Accuracy on movie review dataset using SVM together: %.3f" % (svm_score*100))

Accuracy on movie review dataset using SVM together: 75.203


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn_yhat = neigh.predict(X_test)
knn_score = accuracy_score(knn_yhat, y_test)
print("Accuracy on movie review dataset using KNN together: %.3f" % (knn_score*100))

Accuracy on movie review dataset using KNN together: 62.038


#LPA implementation

### X_Train & Xtest both are being passed while training

In [ ]:
# cell in which train-test both are being passed for training
x_train_test = np.concatenate((X_train, X_test))
no_label_test = [-1 for _ in range(len(y_test))]
y_train_test = np.concatenate((y_train, no_label_test))

In [ ]:
model_all = LabelPropagation(max_iter=4500)
model_all.fit(x_train_test, y_train_test)

/usr/local/lib/python3.8/dist-packages/sklearn/semi_supervised/_label_propagation.py:316: ConvergenceWarning: max_iter=4500 was reached without convergence.
  warnings.warn(


LabelPropagation(max_iter=4500)

In [ ]:
y_pred = model_all.predict(X_test)
score = accuracy_score(y_test, y_pred)
print("Accuracy on movie review dataset from LPA using all data: %.3f" % (score*100))

Accuracy on movie review dataset from LPA using all data: 68.092


## combining semi-supervised and supervised both, use predicted labels in training from LPA in supervised algorithm



LR

In [ ]:
tran_label = model_all.transduction_
tran_label.shape

(10662,)

In [ ]:
from sklearn.linear_model import LogisticRegression
model_all_LR = LogisticRegression(max_iter= 200)
model_all_LR.fit(x_train_test, tran_label)

LogisticRegression(max_iter=200)

In [ ]:
y_pred_LR = model_all_LR.predict(X_test)
score_LR = accuracy_score(y_test, y_pred_LR)
print("Accuracy on movie review dataset using LPA and LR together: %.3f" % (score_LR*100))

Accuracy on movie review dataset using LPA and LR together: 76.562


SVM

In [ ]:
from sklearn.svm import SVC
model_all_svm = SVC()
model_all_svm.fit(x_train_test, tran_label)

SVC()

In [ ]:
y_pred_svm = model_all_svm.predict(X_test)
score_svm = accuracy_score(y_test, y_pred_svm)
print("Accuracy on movie review dataset using LPA and SVM together: %.3f" % (score_svm*100))

Accuracy on movie review dataset using LPA and SVM together: 74.479


# Further splitting train  and not passing Xtest while training, using Xtest only for testing

In [ ]:
# split train into labeled and unlabeled ----> by default, data first shuffled and then divided
X_train_lab, X_train_unlab, y_train_lab, y_train_unlab = train_test_split(X_train, y_train, test_size=0.5, random_state=1, stratify=y_train)

ValueError: ignored

In [ ]:
# summarize training set size
print('Labeled Train Set:', X_train_lab.shape, y_train_lab.shape)
print('Unlabeled Train Set:', X_train_unlab.shape, y_train_unlab.shape)
# summarize test set size
print('Test Set:', X_test.shape, y_test.shape)

In [ ]:
# training input dataset
X_train_mixed = np.concatenate((X_train_lab, X_train_unlab))  
#nolabel for unlabelled data
no_label = [-1 for _ in range(len(y_train_unlab))]
y_train_mixed = np.concatenate((y_train_lab, no_label))
print(X_train_mixed.shape,y_train_mixed.shape)

In [ ]:
# LPA model- default 1000 iterations
model = LabelPropagation(max_iter = 1000)
model.fit(X_train_mixed, y_train_mixed)

In [ ]:
#make predictions on test set
yhat = model.predict(X_test)
score = accuracy_score(y_test, yhat)
print("Accuracy on movie review dataset using LPA: %.3f" % (score*100))

## combining semi-supervised and supervised both, use predicted labels in training from LPA in supervised algorithm



LR

In [ ]:
tran_labels = model.transduction_
tran_labels.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
model_LR = LogisticRegression(max_iter= 200)
model_LR.fit(X_train_mixed, tran_labels)

In [ ]:
yhat_LR = model_LR.predict(X_test)
score_LR = accuracy_score(y_test, yhat_LR)
print("Accuracy on movie review dataset using LPA and LR together: %.3f" % (score_LR*100))

SVM

In [ ]:
from sklearn.svm import SVC
model_svm = SVC()
model_svm.fit(X_train_mixed, tran_labels)

In [ ]:
yhat_svm = model_svm.predict(X_test)
score_svm = accuracy_score(y_test, yhat_svm)
print("Accuracy on movie review dataset using LPA and SVM together: %.3f" % (score_svm*100))